# PiCoGen-2 Colab Notebook (Arabic)
هذا الدفتر مُعد لتشغيل مشروع PiCoGen-2 في Google Colab.

> **تنبيه**: لتفعيل الـGPU في Colab، استخدم شريط القوائم: **Runtime → Change runtime type → Hardware accelerator → GPU** ثم اضغط Save. لا يمكن تحديد GPU من داخل VS Code؛ يجب فعله عبر واجهة Colab.

**ملاحظة هامة:** المشروع يتطلب GPU قوي (يفضل >=16GB VRAM). Colab المجاني ربما لا يكفي لتشغيل النماذج الثقيلة.
استخدم أحد الخيارين أدناه لجلب الشيفرة إلى بيئة Colab: استنساخ من GitHub، رفع ملف ZIP، أو توصيل Google Drive.

## 2) Option A — Clone from GitHub (replace URL)
إذا المستودع عام على GitHub، ضع عنوان المستودع هنا واستنسخ.

In [ ]:
# Clone the repository you provided and enter it
!git clone https://github.com/kareemkamal10/PiCoGen-2.git picogen2_repo
%cd picogen2_repo

# After cloning, install dependencies (may take several minutes)
# Install system packages first if needed, then Python deps:
# !pip install -r requirements.txt
# !pip install -e .

fatal: destination path 'picogen2_repo' already exists and is not an empty directory.
/content/picogen2_repo


: 

In [2]:
# Setup: install essential Python deps (run once) and restart runtime
# Install only light dependencies; skip full requirements that may fail to build
!pip install --upgrade pip
!pip install questionary || true
# optional extras if needed; comment out lines that fail
#!pip install -r requirements.txt || true
#!pip install -e . || true
# Install MPI if SheetSage requires it (optional, safe to run)
!apt-get update -qq && apt-get install -y -qq libopenmpi-dev || true
!pip install mpi4py || true
# Verify installation
!python -c "import questionary, sys; print('questionary ok', questionary.__version__)" || echo 'questionary not available'
# Restart runtime so newly installed packages are importable
import os
print('Restarting runtime to apply installations (this will crash the process)...')
# killing the process forces Colab to restart
os.kill(os.getpid(), 9)

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
questionary ok 2.1.1


: 

: 

### هل يوجد نموذج مُدرب مسبقًا؟
المشروع لا يشمل ملفات نقاط تفتيش كبيرة داخل الريبو. عند استدعاء `PiCoGenDecoder.from_pretrained()` سيحاول البرنامج تنزيل ملفات التكوين والنموذج إلى مجلد التخزين المؤقت (`~/.cache/picogen2`) تلقائيًا عبر `wget` باستخدام عناوين موجودة في `picogen2.assets`.
- إذا استخدمت صورة الـDocker `latest-full` فهي تحتوي على النماذج المحمّلة مسبقًا.
- في Colab، سيُحمّل البرنامج النموذج تلقائيًا عند الحاجة (تأكد أن `wget` متاح).
- التحميل قد يكون كبيرًا ويأخذ وقتًا ومساحة قرصية كبيرة — تحقق من سعة القرص قبل التشغيل.

### هل يوجد نموذج مُدرب مسبقًا؟
المشروع لا يشمل ملفات نقاط تفتيش كبيرة داخل الريبو. عند استدعاء `PiCoGenDecoder.from_pretrained()` سيحاول البرنامج تنزيل ملفات التكوين والنموذج إلى مجلد التخزين المؤقت (`~/.cache/picogen2`) تلقائيًا عبر `wget` باستخدام عناوين موجودة في `picogen2.assets`.
- إذا استخدمت صورة الـDocker `latest-full` فهي تحتوي على النماذج المحمّلة مسبقًا.
- في Colab، سيُحمّل البرنامج النموذج تلقائيًا عند الحاجة (تأكد أن `wget` متاح).
- التحميل قد يكون كبيرًا ويأخذ وقتًا ومساحة قرصية كبيرة — تحقق من سعة القرص قبل التشغيل.

## 4) Install system deps (ffmpeg) and Python deps
بعض الأدوات مثل `ffmpeg` مطلوبة. نفّذ الخلايا التالية بعد أن تكون في مجلد المشروع.

In [ ]:
# Install system packages and Python helpers
!apt-get update -qq && apt-get install -y -qq ffmpeg
# Install pytest (for tests) and other Python tools if needed
!pip install --upgrade pip
!pip install pytest tqdm numpy torch soundfile librosa
# Note: prefer to install from the repo's requirements after cloning/uploading:
# !pip install -r requirements.txt
# !pip install -e .

## 5) Quick smoke test (Python API)
نسخة مبسطة من مثال الـPython في `README.md`. هذه الخلية تحاول تحميل النموذج على `cuda` إن كانت متاحة، وإلا تنبه.

In [ ]:
# Quick smoke test (adjust paths if needed)
import torch
print('CUDA available:', torch.cuda.is_available())
try:
    import picogen2
    from picogen2 import assets
    print('picogen2 imported OK')
    # Try to access the test song path if package provides it
    try:
        audio_file = assets.test_song()
        print('Found test audio:', audio_file)
    except Exception as e:
        print('No test asset available or error:', e)
except Exception as e:
    print('Import error — ensure you ran pip install -e . and dependencies:', e)

## رفع الأغنية وتشغيل التحويل (قد يستغرق وقتًا)
أولًا ارفع ملف MP3 عبر الواجهة، ثم شغّل الخلية التالية لتحويله إلى ملف ميدي Piano.

In [ ]:
# Upload a single MP3 file (browser picker)
from google.colab import files
uploaded = files.upload()
if len(uploaded) == 0:
    raise SystemExit('No file uploaded')
fname = next(iter(uploaded))
print('Uploaded:', fname)
import shutil, os
os.makedirs('/content', exist_ok=True)
shutil.copy(fname, '/content/' + fname)
audio_path = '/content/' + fname
output_dir = '/content/output'
os.makedirs(output_dir, exist_ok=True)
print('audio_path=', audio_path)

In [ ]:
# Convert uploaded MP3 to MIDI (may take several minutes and requires GPU)
import json, numpy as np, torch
from pathlib import Path
print('CUDA available:', torch.cuda.is_available())
# ensure wget available for downloading pretrained checkpoint if needed
import os
if os.system('which wget > /dev/null') != 0:
    os.system('apt-get update -qq && apt-get install -y wget')
# Import project API (assumes repo already cloned and installed)
import picogen2
from picogen2.model import PiCoGenDecoder
from picogen2.infer import detect_beat, extract_sheetsage_feature, decode
from picogen2 import Tokenizer

audio_file = Path(audio_path)
out_dir = Path(output_dir)
# Load tokenizer and model (this will download pretrained files if not present)
tokenizer = picogen2.Tokenizer()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = PiCoGenDecoder.from_pretrained(device=device)

# Detect beats
beat_file = out_dir / 'beat.json'
detect_beat(audio_file, beat_file)

# Extract features via SheetSage (may be slow)
feature_file = out_dir / 'sheetsage.npz'
extract_sheetsage_feature(audio_file, feature_file, beat_file)
d = np.load(feature_file)
melody = d['melody']
harmony = d['harmony']
beat_info = json.loads(beat_file.read_text())

# Decode to events and save MIDI
out_events = decode(model, tokenizer, beat_info, melody, harmony, device=device)
(out_dir / 'piano.txt').write_text('\n'.join(map(str, out_events)))
tokenizer.events_to_midi(out_events).dump(out_dir / 'piano.mid')
print('Saved MIDI to:', out_dir / 'piano.mid')
# Offer download
from google.colab import files as _files
_files.download(str(out_dir / 'piano.mid'))

## 6) Run demo script (example)
إذا استخدمت الاستنساخ من GitHub أو رفعت المستودع، يمكنك تشغيل السكربت `infer` أو `demo.sh` كما في README.
أو استعمل الواجهة البرمجية المذكورة في README لتشغيل وظيفة `picogen2.decode`.

---
## ملاحظات ختامية
- هذا المشروع مُصمم ليعمل على Linux مع GPU كبير الذاكرة.
- إن كانت موارد Colab غير كافية، أوصي بتشغيله على VM سحابي (مثل Google Cloud, AWS, أو Paperspace) مع GPU مناسبة.
- هل تريد أن أضع ملف `requirements.txt` داخل الخلية ليُثبت مباشرة أو أُعد نسخة جاهزة من الأوامر مع رابط استنساخ؟